In [ ]:
###########Script pour créer la base de données, exécuter les requêtes SQL dans le conteneur Docker, extraire les résultats et les sauvegarder dans des fichiers CSV.###########

In [50]:
import requests
import os
import gzip
from tqdm.notebook import trange
import subprocess


In [51]:
# Demarrer le conteneur Docker pour MySQL
print("🚀 Starting Docker container...")
COMPOSE_FILE="../docker/docker-compose.yml"
subprocess.run(["docker", "compose", "-f", COMPOSE_FILE, "up", "-d", "--build"])


🚀 Starting Docker container...


 Image docker-mysql Building 


#1 [internal] load local bake definitions
#1 reading from stdin 558B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile.mysql
#2 transferring dockerfile: 285B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/mysql:8.0
#3 ...

#4 [auth] library/mysql:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/mysql:8.0
#3 DONE 1.0s

#5 [internal] load .dockerignore
#5 transferring context: 2B done
#5 DONE 0.0s

#6 [1/1] FROM docker.io/library/mysql:8.0@sha256:99d774bf02a48a1bb1c599920d2571946d31e5940b854b02737d5e95c184358f
#6 resolve docker.io/library/mysql:8.0@sha256:99d774bf02a48a1bb1c599920d2571946d31e5940b854b02737d5e95c184358f done
#6 CACHED

#7 exporting to image
#7 exporting layers done
#7 exporting manifest sha256:cc0547b9a74fe963a4486a74cf94975e891023bb7340e940939d9030dc7715c7 done
#7 exporting config sha256:c5bfc23de452870463110aea77e9e84417f5e2bb10e662033d28b4d55c2c6b54 done
#7 exporting attestati

 Image docker-mysql Built 
 Network docker_default Creating 
 Network docker_default Created 
 Container moviesdb_mysql Creating 
 Container moviesdb_mysql Created 
 Container moviesdb_mysql Starting 
 Container moviesdb_mysql Started 


CompletedProcess(args=['docker', 'compose', '-f', '../docker/docker-compose.yml', 'up', '-d', '--build'], returncode=0)

In [106]:
#Modifier le nom du fichier pour chaque requête SQL
#Copier le fichier de requête dans le conteneur
file_name = "marvel_directors.sql"

print("📁 Copying sql file into docker container...")
result = subprocess.run(["docker", "cp", f"../mysql/{file_name}", f"moviesdb_mysql:/tmp/{file_name}"], capture_output=True, text=True, check=False)
print(result.stderr)

📁 Copying sql file into docker container...



In [107]:
# Executer la requête et exporter le CSV
output_name = file_name.replace('.sql', '.csv')
CONTAINER = "moviesdb_mysql"
SQL_IN_CONTAINER = f"/tmp/{file_name}"  # path inside the container where the SQL file is located
DB = "IMDb"  # database name inside the MySQL container
HOST_OUT_PATH = f"../data/tests/{output_name}"
CONTAINER_OUT = f"/var/lib/mysql-files/{output_name}"  # full path inside the container for the output CSV

#supprimer le fichier csv s'il existe déjà
result = subprocess.run([
    "docker", "exec", "-i", CONTAINER,
    "rm", "-f", CONTAINER_OUT
], check=True)

#executer la requete sql 
result = subprocess.run([
    'docker', 'exec',
    CONTAINER,
    'bash', '-c',
    f'mysql -u root -proot IMDb < {SQL_IN_CONTAINER}'
], capture_output=True, text=True, check=False)

print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)
print("Return code:", result.returncode)

if result.returncode != 0:
    print("❌ Docker cp failed!")
else:
    print("✅ File copied successfully")


STDOUT: 
STDERR: mysql: [Warning] Using a password on the command line interface can be insecure.

Return code: 0
✅ File copied successfully


In [108]:
#Copier le fichier dans le dossier local
result = subprocess.run([
    'docker', 'cp',
    f'{CONTAINER}:{CONTAINER_OUT}',
    HOST_OUT_PATH
], capture_output=True, text=True, check=False)

if result.returncode != 0:
    print("❌ Docker cp failed!")
    print("STDERR:", result.stderr)
else:
    print("✅ File copied successfully")

✅ File copied successfully


In [109]:
#Lire le CSV avec pandas
import pandas as pd

orig_path = f"../data/tests/{output_name}"

# read raw bytes and try common encodings, fallback to latin-1 with replacement
# with open(orig_path, 'rb') as f:
#     raw = f.read()
#handle malformed json in character_name column by extracting first character from JSON array
import json
def extract_first_character(x):
    """Extract first character from JSON array, handle JSON decode errors gracefully"""
    if pd.notnull(x) and x != "\\N" and str(x).strip():
        try:
            parsed = json.loads(x)
            return parsed[0] if isinstance(parsed, list) and len(parsed) > 0 else x
        except (json.JSONDecodeError, ValueError, TypeError):
            # If JSON parsing fails, return the value as-is
            return x
    return None
columns_movies = ["title_id","primary_title","genres","start_year"]
columns_actors = ["person_id", "person_name"]
columns_directors = ["title_id", "person_id", "person_name", "job", "category"]
columns_characters = ["character_name"]

if "movies" in output_name:
    marvel_df = pd.read_csv(orig_path, sep=",",
    header=0,
    encoding="utf-8",
    engine="python", names = columns_movies)

elif "characters" in output_name: 
    marvel_df = pd.read_csv(orig_path, encoding='latin-1', header=0,   names= columns_characters)
    marvel_df.iloc["character_name"] = marvel_df.iloc["character_name"].apply(extract_first_character)
    # marvel_df["character_name"] = marvel_df["character_name"].str.replace(r"[\[\]'\"]", "", regex=True)
    
elif "directors" in output_name:
    marvel_df = pd.read_csv(orig_path, sep=",",
    header=0,
    encoding="utf-8",
    engine="python", names= columns_directors)

elif "actors" in output_name:
    marvel_df = pd.read_csv(orig_path, sep=",",
    header=0,
    encoding="utf-8",
    engine="python", names= columns_actors)

print(marvel_df.head())
print(marvel_df.shape)
print("Returned rows:", len(marvel_df))
marvel_df.to_csv("../data/tests/"+output_name, index=False)



     title_id  person_id        person_name       job  category
0   tt5463162  nm0005351      Ryan Reynolds  producer  producer
1   tt1431045  nm0005351      Ryan Reynolds  producer  producer
2  tt22084616  nm0032696           Avi Arad  producer  producer
3   tt0371746  nm0032696           Avi Arad  producer  producer
4   tt5095030  nm0112780  Stephen Broussard  producer  producer
(107, 5)
Returned rows: 107


In [94]:
# #Arreter le conteneur Docker

print("🚀 Stopping Docker container")
COMPOSE_FILE="../docker/docker-compose.yml"
CONTAINER_NAME="moviesdb_mysql"

subprocess.run(["docker", "stop", CONTAINER_NAME])
subprocess.run(["docker", "compose", "-f", COMPOSE_FILE, "down"])

🚀 Stopping Docker container
moviesdb_mysql


 Container moviesdb_mysql Stopping 
 Container moviesdb_mysql Stopped 
 Container moviesdb_mysql Removing 
 Container moviesdb_mysql Removed 
 Network docker_default Removing 
 Network docker_default Removed 


CompletedProcess(args=['docker', 'compose', '-f', '../docker/docker-compose.yml', 'down'], returncode=0)